In [39]:
! conda install -c conda-forge folium --yes
! pip install emoji --upgrade

Solving environment: done

# All requested packages already installed.

     |████████████████████████████████| 51kB 1.7MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/86/09/26/f944015841423cd516e8a97f30e29be59e53461aea8b7d3458
Successfully built emoji


In [40]:
# standard weapons of choice
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelBinarizer, CategoricalEncoder, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans

# webscraping weapons of choice for standard HTMl sites
import requests
from bs4 import BeautifulSoup

# geolocation helper
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# all the viz
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from tabulate import tabulate
# import fontawesome as fa
import emoji

%matplotlib inline

##  <center>🍁 Toronto Neighborhood Exploration 🍁</center>

#### <center>Let's explore some neighborhoods in Toronto, Ontario Canada, cluster them together, and then find cool stuff in these neighborhoods by using the FourSquare API</center>

### Let's get some data in here by scraping what we need from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [46]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup= BeautifulSoup(response.content, 'lxml')
toronto_table = soup.find_all('table')[0]
toronto_df = pd.read_html(str(toronto_table))[0]
toronto_df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


#### Lets get rid all the postcodes that don't have boroughs assciated wtih them. 